In [ ]:
import dask.dataframe as dd
# Read the CSV file into a DataFrame
df = dd.read_csv('/content/drive/MyDrive/goodreads_interactions.csv')
df.head()

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [ ]:
%%writefile data.yaml
file_type: csv
file_name: goodreads_interactions
columns:
  user_id,
  book_id,
  is_read,
  rating,
  is_reviewed

Writing data.yaml


In [ ]:
  import yaml
  #import testutility as util
  with open('data.yaml', 'r') as file:
    data = yaml.load(file, Loader=yaml.FullLoader)

  print(data['file_type'])

csv


In [ ]:
  def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

  def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [ ]:
import os
import gzip
expected_columns = data['columns']
expected_num_columns = len(expected_columns)

if len(df.columns) != expected_num_columns:
    print(f"Number of columns does not match. Expected: {expected_num_columns}, Actual: {len(df.columns)}")

# Validate column names
if not all(col in df.columns for col in expected_columns):
    print("Column names do not match the expected names.")

output_file = 'output_file.txt.gz'
df.to_csv(output_file, sep='|', index=False, compression='gzip')

total_rows = len(df)
total_columns = len(df.columns)
file_size = os.path.getsize(output_file)

print(f"Summary:")
print(f"Total number of rows: {total_rows}")
print(f"Total number of columns: {total_columns}")
print(f"File size: {file_size} bytes")

Number of columns does not match. Expected: 46, Actual: 5
Column names do not match the expected names.
Summary:
Total number of rows: 228648342
Total number of columns: 5
File size: 4096 bytes


In [ ]:
from google.colab import files
import shutil

# Define the path to the folder you want to download
folder_path = '/content/output_file.txt.gz'

# Define the name of the ZIP file
zip_file = '/content/output.zip'

# Compress the folder into a ZIP file
#shutil.make_archive(zip_file.split('.')[0], 'zip', folder_path)

# Download the ZIP file to your local machine
#files.download(zip_file)